In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, when, first, avg
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from functools import reduce
from pyspark.sql import DataFrame

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# Carregar tabelas
time_df = spark.table("database.time")
employment_df = spark.table("database.gender")

# Lista de países e anos
paises = ["%Germany%", "%Switzerland%", "%Poland%", "%Hungary%"]
anos = ["2010", "2013"]

# Filtrando os DataFrames

def filter_by_multiple_conditions(df: DataFrame, paises: list, anos: list) -> DataFrame:
    conditions = reduce(lambda a, b: a | b, (col("nome_pais").like(p) for p in paises))
    df = df.filter(conditions)
    df = df.filter(col("ano").isin(anos))
    return df

time_df = filter_by_multiple_conditions(time_df, paises, anos)
employment_df = filter_by_multiple_conditions(employment_df, paises, anos)

time_df = time_df.drop("cod_indicador", "cod_pais")

employment_df = employment_df.drop("Country_code", "Country_name", "Indicator_name", "Indicator_code", "cod_indicador", "cod_pais")

# Filtrar pelos indicadores de emprego
employment_df = employment_df.filter(F.col("nome_indicador").like("%Employment to population ratio%"))

# Adicionar a coluna "genero" com base no nome do indicador
employment_df = employment_df.withColumn("desagregacao",
    F.when(F.col("nome_indicador").like("% female%"), "female")
    .when(F.col("nome_indicador").like("% male%"), "male")
)
employment_df = employment_df.filter(F.col("desagregacao").isNotNull())

# Calcular a média dos valores por ano, gênero e indicador
employment_df_media = employment_df.groupBy("nome_pais", "ano", "desagregacao", "nome_indicador") \
    .agg(F.avg("valor").alias("valor"))

# Selecionar apenas uma linha por ano e gênero
window = Window.partitionBy("nome_pais", "ano", "desagregacao").orderBy(F.col("valor"))

employment_df_final = employment_df_media.withColumn("row_number", F.row_number().over(window)) \
    .filter(F.col("row_number") == 1).drop("row_number")

columns_order = ["nome_indicador", "nome_pais", "ano", "valor", "desagregacao"]

# Reorganizando as colunas no DataFrame employment_df_final
employment_df_final = employment_df_final.select(columns_order)

In [3]:
time_df = time_df.withColumn("valor", col("valor").cast("float"))
employment_df_final = employment_df_final.withColumn("valor", col("valor").cast("float"))

In [4]:
df_concatenado = employment_df_final.union(time_df)

In [5]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

df_concatenado.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaEmploymentGold")

In [6]:
spark.sql(
    """
    SELECT * FROM database.EmploymentGold
    """
).show(500)

+-----------+--------------------+----+--------+------------+
|  nome_pais|      nome_indicador| ano|   valor|desagregacao|
+-----------+--------------------+----+--------+------------+
|    Germany|Employment to pop...|2010|   44.48|      female|
|    Germany|Employment to pop...|2010|   47.65|        male|
|    Germany|Employment to pop...|2013|   45.22|      female|
|    Germany|Employment to pop...|2013|  48.133|        male|
|    Hungary|Employment to pop...|2010|   16.55|      female|
|    Hungary|Employment to pop...|2010|   19.89|        male|
|    Hungary|Employment to pop...|2013|   17.03|      female|
|    Hungary|Employment to pop...|2013|   23.05|        male|
|     Poland|Employment to pop...|2010|   22.09|      female|
|     Poland|Employment to pop...|2010|  30.476|        male|
|     Poland|Employment to pop...|2013|   19.51|      female|
|     Poland|Employment to pop...|2013|   28.64|        male|
|Switzerland|Employment to pop...|2010|   56.85|      female|
|Switzer

In [7]:
spark.stop()